In [1]:
!pip install scikit-learn

In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [4]:
data = pd.read_csv('/content/drive/MyDrive/PROJECT/Dataset/train.csv', parse_dates=['Order Date'], dayfirst=True)

In [5]:
data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,2017-11-08,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,2017-06-12,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [14]:
sales_by_category = data.groupby(["Category", data["Order Date"].dt.year])["Sales"].sum().reset_index()
forecasted_sales = pd.DataFrame(columns=["Category", "Forecasted_sales_2019"])
target_year = 2019
result = []

for category in sales_by_category["Category"].unique():
    category_data = sales_by_category[sales_by_category["Category"] == category]
    category_data["Order Date"] = pd.to_datetime(category_data["Order Date"])
    category_data["Year"] = category_data["Order Date"].dt.year
    scaler = MinMaxScaler()

    features = ["Sales"]
    target = "Sales"

    category_data["Sales"] = scaler.fit_transform(category_data["Sales"].values.reshape(-1,1))

    train_data, valid_data = train_test_split(category_data, test_size=0.2, random_state=42, shuffle=False)

    X = train_data[features].values
    y = train_data[target].values
    #LSTM accepts input in the shape of (N*T*M) ie 3D
    X = X.reshape(X.shape[0], X.shape[1], 1)

    model = Sequential()
    model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1))

    model.compile(loss='mse', optimizer='adam')
    model.fit(X, y, epochs=100, batch_size=16, verbose=0)

    forecast_data = np.array([target_year])
    #because this will be the input feature to predict sale so reshape
    forecast_data = forecast_data.reshape(1,1,1)
    forecast = model.predict(forecast_data)
    forecast = scaler.inverse_transform(forecast.reshape(-1,1))[0][0]

    forecasted_sales = pd.concat([forecasted_sales, pd.DataFrame({
        "Category": [category],
        "Forecasted_sales_2019": [forecast]
    })], ignore_index=True)

    X_val = valid_data[features].values
    X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)  # Reshape to (N * T * M)

    true_values = valid_data[target]
    predicted_values = model.predict(X_val)
    mae = mean_absolute_error(true_values, predicted_values)
    result.append({
        "Category": category,
        'True Values': true_values.values,
        'Predicted Values': predicted_values,
        "MAE": mae,
    })
    for i in result:
        print(f"Category: {i['Category']}")
        print(f"True Values: {i['True Values']}")
        print(f"Predicted Values: {i['Predicted Values']}")
        print(f"MAE: {i['MAE']}")
        print()

forecasted_sales["Forecasted_sales_2019"] = forecasted_sales["Forecasted_sales_2019"].apply(lambda x: '{:,.2f}'.format(x))
print("Forecasted Sales in 2019 for Each Product Category")
print(forecasted_sales)

<ipython-input-14-e0071a261b96>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data["Order Date"] = pd.to_datetime(category_data["Order Date"])
<ipython-input-14-e0071a261b96>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data["Year"] = category_data["Order Date"].dt.year
<ipython-input-14-e0071a261b96>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

1/1 [==============================] - 0s 451ms/step
Category: Furniture
True Values: [1.]
Predicted Values: [[0.7241143]]
MAE: 0.2758857011795044



<ipython-input-14-e0071a261b96>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data["Order Date"] = pd.to_datetime(category_data["Order Date"])
<ipython-input-14-e0071a261b96>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data["Year"] = category_data["Order Date"].dt.year
<ipython-input-14-e0071a261b96>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

1/1 [==============================] - 1s 514ms/step
Category: Furniture
True Values: [1.]
Predicted Values: [[0.7241143]]
MAE: 0.2758857011795044

Category: Office Supplies
True Values: [1.]
Predicted Values: [[0.57377636]]
MAE: 0.42622363567352295



<ipython-input-14-e0071a261b96>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data["Order Date"] = pd.to_datetime(category_data["Order Date"])
<ipython-input-14-e0071a261b96>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data["Year"] = category_data["Order Date"].dt.year
<ipython-input-14-e0071a261b96>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

1/1 [==============================] - 0s 400ms/step
Category: Furniture
True Values: [1.]
Predicted Values: [[0.7241143]]
MAE: 0.2758857011795044

Category: Office Supplies
True Values: [1.]
Predicted Values: [[0.57377636]]
MAE: 0.42622363567352295

Category: Technology
True Values: [1.]
Predicted Values: [[0.6959503]]
MAE: 0.3040496706962583

Forecasted Sales in 2019 for Each Product Category
          Category Forecasted_sales_2019
0        Furniture            810,846.44
1  Office Supplies          1,178,013.12
2       Technology          1,323,970.88


##Why we choose Sales as feature and not Year
LSTM models are automatically designed to learn temporal patterns from sequential data making it suitable for direct forecasting based on the inherent patterns within the historical time series